In [ ]:
import pandas as pd
import numpy as np
from gensim.models import word2vec
import gensim
import nbimporter
from pre_processamento import *
from features_media import *
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
import numpy as np
import sklearn.metrics as skm
import glob
%matplotlib inline

In [ ]:
#Classificadores
from sklearn import naive_bayes
from sklearn import svm

In [ ]:
minWordCount = 1
removeStopWords = True
numFeatures = 300

In [ ]:
fileName = "IPHONE6"

In [ ]:
# Carrega dataset e realiza shuffle
dataset = pd.read_csv("Data/Bases/{}.csv".format(fileName), delimiter=',', header=0)
dataset.shape

In [ ]:
# Instancia um novo pré-processador
preprocessor_classifier = PreProcessor(remove_stopwords=True, word_min_count=minWordCount, stemming=True)

# Manda um conjunto de textos para ser pré-processado
preprocessor_classifier.fit(dataset["TEXT"])

In [ ]:
cleaned_tweets_word_list = preprocessor_classifier.get_cleaned_sentences_word_list()

In [ ]:
classifiers = [
    ("NBB", naive_bayes.BernoulliNB()),
    ("LinearSVM", svm.LinearSVC())
]

In [ ]:
resultados = []

for fileModel in glob.glob("Data/Grupos/{}/*".format(fileName)):
    modelW2V = word2vec.Word2Vec.load(fileModel)
    # Instancia o contrutor do vetor
    generator = FeatureGeneratorMedia(cleaned_tweets_word_list, modelW2V, numFeatures)

    # Gera os vetores para cada tweet
    generator.gen_features_vec()

    # Pega os vetores gerados
    features_w2v = generator.get_features_vec()
    
    for (c_name, c) in classifiers:
        line = [fileModel]
        y_pred = cross_val_predict(c, features_w2v, dataset['CLASS'], cv=5)
        line.append(c_name)
        line.append(skm.accuracy_score(y_pred=y_pred, y_true=dataset['CLASS']))
        line.append(skm.precision_score(y_pred=y_pred, y_true=dataset['CLASS']))
        line.append(skm.recall_score(y_pred=y_pred, y_true=dataset['CLASS']))
        line.append(skm.f1_score(y_pred=y_pred, y_true=dataset['CLASS']))
        resultados.append(line)

In [ ]:
np.savetxt("Resultados/{}_media.csv".format(fileName), np.array(resultados), delimiter=',', fmt="%s")